# This is going to be my first ever EDA kernel

**I will gradually update this kernel**

<h1 style="color:blue;">
If you find this kernel interesting, please drop an  <font color="red">UPVOTE</font>. It motivates me to produce more quality content :)
</h1>

# what to do in this competition?

**The dataset contains photos of streets, taken from the roof of a car. We're attempting to predict the position and orientation of all un-masked cars in the test images. we should also provide a confidence score indicating how sure we are of your prediction.**

In [ ]:


!pip install -U git+https://github.com/albu/albumentations



**Imports**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
from mpl_toolkits.mplot3d import Axes3D
import scipy.ndimage as ndimage
import cv2
from tqdm import tqdm#_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from functools import reduce
import os
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import cv2
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import models
from torchvision import transforms, utils

from albumentations import Compose, ShiftScaleRotate, RandomCrop, \
    ToGray, CLAHE, GaussNoise, RandomGamma, \
    RandomBrightnessContrast, RGBShift, HueSaturationValue,Normalize
from albumentations import *
from albumentations.pytorch import ToTensor


from tqdm import tqdm_notebook

PATH = '../input/pku-autonomous-driving/'
os.listdir(PATH)
from skimage.io import imread
import os
from math import sin, cos

In [ ]:
DATASET_DIR = '/kaggle/input/pku-autonomous-driving/'

In [ ]:
from albumentations import (HorizontalFlip,OpticalDistortion,VerticalFlip,GridDistortion,RandomBrightnessContrast,OneOf,ElasticTransform,RandomGamma,IAAEmboss,Blur,RandomRotate90,Transpose, ShiftScaleRotate, Normalize, Resize, Compose, GaussNoise)

> Officer: Sir have you been drinking?

> “Of course, but not the car” ;)

**Before we begin,lets watch a video first**

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/1W9q5SjaJTc"rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

It can be seen in the video above - on the front camera of that Self-Driving Car we have **drive net** detecting obstacles the bounding boxes around the cars we have **weight net** detecting the intersection the yellow box around everything, *weight net*  then also detecting traffic lights and traffic signs and light that is classifying traffic light staid correctly is red we also have sign classification going on using **sign net** at the same time *drive net* is detecting pedestrians in the cyan bounding boxes on the far side of the intersection we also have **open road net** tracing out the free space around obstacles on the scene and on top of that we have our object tracking from frame to frame you see the track IDs on the top of each bounding box we also have our **camera base DNN distance estimation** running so you see the distance and metres displayed at the bottom of each box.**clear sight net** is also running in the background assessing whether and how well the cameras can see in our four cameras around perception set up on our embedded EDX platform and all of these rich perception functionality is what our planning and control software is going to use to execute the autonomous driving maneuvers that you will see if you see the video above.



**I would also like you to recommend watching another video**

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400"  src="https://www.youtube.com/embed/GlbZA62eXpU" rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

**From the video above you can see Makers of Baidu's new driverless car believe it's safer than a human driver and hope to have more on the street within the next 10 years. CNN's Matt Rivers reports.**

In [ ]:
print(os.listdir(PATH))

**Directories**

In [ ]:
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'sample_submission.csv')

**size of test set**

In [ ]:
test_size = len(test)
test_size

**public lb uses 9% of 2021 samples means**

In [ ]:
public_test = int(test_size * 0.09)

In [ ]:
print(public_test,'samples for testing public lb!!')

*181 samples for for public lb is very very low so DON"T TRUST YOUR PUBLIC LB SCORE AT ALL!*

**Converting ID_60565cf6d.jpg image from train_images directory to 3D**

*the code below shows the image as a surface in 3D using MatPlotLib, numpy, and ndimage (for filtering).If the image is grayscale, the gray values of each pixel can determine the height of the surface.It may be necessary to blur (i.e. filter) the image to smooth out spikes that will occur due to adjacent pixels with very different values.The meshgrid function is used to set up a mesh for the surface. The ‘[::-1]’ reverses the meshgrid rows so the surface has the same orientation as the image.*

[source link](https://www.quora.com/How-do-I-convert-a-single-2D-image-to-3D-using-python)

In [ ]:
imageFile =  PATH + 'train_images/ID_60565cf6d.jpg'
mat = imread(imageFile)
mat = mat[:,:,0] # get the first channel
rows, cols = mat.shape
xv, yv = np.meshgrid(range(cols), range(rows)[::-1])
     
blurred = ndimage.gaussian_filter(mat, sigma=(5, 5), order=0)
fig = plt.figure(figsize=(40,40))
     
ax = fig.add_subplot(221)
ax.imshow(mat, cmap='gray')
     
ax = fig.add_subplot(222, projection='3d')
ax.elev= 75
ax.plot_surface(xv, yv, mat)
     
ax = fig.add_subplot(223)
ax.imshow(blurred, cmap='gray')
     
ax = fig.add_subplot(224, projection='3d')
ax.elev= 75
ax.plot_surface(xv, yv, blurred)
plt.show()

**Lets visualize a messy mask from test_masks directory**

In [ ]:
imageFile =  PATH + 'test_masks/ID_f64a00c44.jpg'
mat = imread(imageFile)
mat = mat[:,:,0] # get the first channel
rows, cols = mat.shape
xv, yv = np.meshgrid(range(cols), range(rows)[::-1])
     
blurred = ndimage.gaussian_filter(mat, sigma=(5, 5), order=0)
fig = plt.figure(figsize=(50,50))
     
ax = fig.add_subplot(221)
ax.imshow(mat, cmap='gray')
     
ax = fig.add_subplot(222, projection='3d')
ax.elev= 75
ax.plot_surface(xv, yv, mat)
     
ax = fig.add_subplot(223)
ax.imshow(blurred, cmap='gray')
     
ax = fig.add_subplot(224, projection='3d')
ax.elev= 75
ax.plot_surface(xv, yv, blurred)
plt.show()

# Means and stds of single image

In [ ]:
imageFile =  PATH + 'train_images/ID_01021ce21.jpg'
mat = imread(imageFile)

from numpy import asarray
from PIL import Image
# load image
#image = mat #Image.open('sydney_bridge.jpg')
#image = Image.open(PATH + 'train_images/ID_60565cf6d.jpg')
image = cv2.imread(PATH + 'train_images/ID_60565cf6d.jpg')
image = cv2.resize(image, (1536, 512))    
#print(image.shape)
pixels = asarray(image)
# convert from integers to floats
pixels = pixels.astype('float32')
# calculate per-channel means and standard deviations
means = pixels.mean(axis=(0,1), dtype='float64')
stds = pixels.std(axis=(0,1), dtype='float64')
print('Means: %s, Stds: %s' % (means, stds))
# per-channel standardization of pixels
pixels = (pixels - means) / stds
# confirm it had the desired effect
means = pixels.mean(axis=(0,1), dtype='float64')
stds = pixels.std(axis=(0,1), dtype='float64')
print('Means: %s, Stds: %s' % (means, stds))

In [ ]:
image = cv2.imread(PATH + 'train_images/ID_60565cf6d.jpg').astype(np.float32) / 255
img = image
img = cv2.resize(img, (1536, 512))
print(image.shape)
print(image.mean())
print(image.std())

img -= img.mean()
img /= img.std()
print('-------------------------------')
print(img.shape)
print(img.mean())
print(img.std())

# means and stds (samples vs pixels) from train images directory

**stolen from here :** https://www.kaggle.com/phunghieu/cloud-mean-std-calculation

In [ ]:
h = 1536
w = 512


In [ ]:
def compute_sample_mean(im_dir, image_files):
    """
    Parameters:
        im_dir: The directory that contains images.
        image_files: List of image files you need.

    Returns:
        sample-mean
    """

    if len(image_files) > 0:
        image_sum = cv2.imread(os.path.join(im_dir, image_files[0]))
        image_sum = cv2.resize(image_sum, (h, w))  
        image_sum = cv2.cvtColor(image_sum, cv2.COLOR_BGR2RGB).astype(np.float64)

        for image_file in tqdm_notebook(image_files[1:]):
            img = cv2.imread(os.path.join(im_dir, image_file))
            img = cv2.resize(img, (h, w))  
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            image_sum += img

        return image_sum/(255*len(image_files))

    return None


def compute_pixel_mean(im_dir, image_files, sample_mean=None):
    """
    Parameters:
        im_dir: The directory that contains images.
        image_files: List of image files you need.
        sample_mean: sample-mean value.

    Returns:
        pixel-mean
    """

    if sample_mean is None:
        sample_mean = compute_sample_mean(im_dir, image_files)

    return np.mean(sample_mean, axis=(0, 1))


def compute_sample_std(im_dir, image_files, sample_mean=None):
    """
    Parameters:
        im_dir: The directory that contains images.
        image_files: List of image files you need.
        sample_mean: sample-mean value.

    Returns:
        sample-std
    """

    if len(image_files) > 0:
        if sample_mean is None:
            sample_mean = compute_sample_mean(im_dir, image_files)

        square_diff_sum = 0
        for image_file in tqdm_notebook(image_files):
            img = cv2.imread(os.path.join(im_dir, image_file))
            img = cv2.resize(img, (h, w))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float64)
            img /= 255.
            square_diff_sum += np.square(np.abs(img - sample_mean))

        return np.sqrt(square_diff_sum/len(image_files))

    return 0


def compute_pixel_std(im_dir, image_files, sample_mean=None, pixel_mean=None):
    """
    Parameters:
        im_dir: The directory that contains images.
        image_files: List of image files you need.
        sample_mean: sample-mean value.
        pixel_mean: pixel-mean value.

    Returns:
        pixel-std
    """

    if len(image_files) > 0:
        if pixel_mean is None:
            if sample_mean is not None:
                pixel_mean = compute_pixel_mean(im_dir, image_files, sample_mean)
            else:
                pixel_mean = compute_pixel_mean(im_dir, image_files)

        square_diff_sum = 0
        for image_file in tqdm_notebook(image_files):
            img = cv2.imread(os.path.join(im_dir, image_file))
            img = cv2.resize(img, (h, w))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float64)
            img /= 255.
            square_diff_sum += np.sum(np.square(np.abs(img - pixel_mean)), axis=(0, 1))

        return np.sqrt(square_diff_sum/(len(image_files)*img.shape[0]*img.shape[1]))

    return 0

In [ ]:
train_images_dir = os.path.join(DATASET_DIR, 'train_images')
image_files = os.listdir(train_images_dir)

In [ ]:
sample_mean = compute_sample_mean(train_images_dir, image_files)
sample_std = compute_sample_std(train_images_dir, image_files, sample_mean=sample_mean)
pixel_mean = compute_pixel_mean(train_images_dir, image_files, sample_mean=sample_mean)
pixel_std = compute_pixel_std(train_images_dir, image_files, pixel_mean=pixel_mean)

In [ ]:
print('Sample mean:', sample_mean)

In [ ]:
print('Sample std:', sample_std)

In [ ]:
print('Pixel mean:', pixel_mean)
print('Pixel std:', pixel_std)

# means and stds (samples vs pixels) from test images directory

In [ ]:
train_images_dir = os.path.join(DATASET_DIR, 'test_images')
image_files = os.listdir(train_images_dir)

In [ ]:
sample_mean = compute_sample_mean(train_images_dir, image_files)
sample_std = compute_sample_std(train_images_dir, image_files, sample_mean=sample_mean)
pixel_mean = compute_pixel_mean(train_images_dir, image_files, sample_mean=sample_mean)
pixel_std = compute_pixel_std(train_images_dir, image_files, pixel_mean=pixel_mean)

In [ ]:
print('Sample mean:', sample_mean)

In [ ]:
print('Sample std:', sample_std)

In [ ]:
print('Pixel mean:', pixel_mean)
print('Pixel std:', pixel_std)

**Now lets visualize a good looking one from test_masks directory**

In [ ]:
imageFile =  PATH + 'test_masks/ID_d17f58f89.jpg'
mat = imread(imageFile)
mat = mat[:,:,0] # get the first channel
rows, cols = mat.shape
xv, yv = np.meshgrid(range(cols), range(rows)[::-1])
     
blurred = ndimage.gaussian_filter(mat, sigma=(5, 5), order=0)
fig = plt.figure(figsize=(50,50))
     
ax = fig.add_subplot(221)
ax.imshow(mat, cmap='gray')
     
ax = fig.add_subplot(222, projection='3d')
ax.elev= 75
ax.plot_surface(xv, yv, mat)
     
ax = fig.add_subplot(223)
ax.imshow(blurred, cmap='gray')
     
ax = fig.add_subplot(224, projection='3d')
ax.elev= 75
ax.plot_surface(xv, yv, blurred)
plt.show()

* > COMPARE OUTPUT OF ABOVE 2 IMAGES(messy mask and good looking mask) on your own!


# Expanding out the prediction string for the first vehicle

We know the order of each value in the prediction string. We can expand it out for the first vehicle and see some statistics for this first vehicle position.


In [ ]:
# taken from : https://www.kaggle.com/robikscube/autonomous-driving-introduction-data-review

train_expanded = pd.concat([train, train['PredictionString'].str.split(' ', expand=True)], axis=1)
train_expanded = train_expanded.rename(columns={0 : '1_model_type', 1 : '1_yaw', 2 : '1_pitch',
                                                3 : '1_roll', 4 : '1_x', 5 : '1_y', 6 : '1_z'})
train_expanded.drop('PredictionString', axis=1).head()

**I have borrowed some code from this beautiful kernel[ Lyft Competition : Understanding the data](https://www.kaggle.com/tarunpaparaju/lyft-competition-understanding-the-data) for generating few plots**

# KDE Plot


In [ ]:
new_train_objects = train_expanded
plot = sns.jointplot(x=new_train_objects['1_x'][:1000], y=new_train_objects['1_y'][:1000], kind='kde', color='blueviolet')
plot.set_axis_labels('1_x', '1_y', fontsize=16)
plt.show()

**Converting center_x,center_y,center_z,roll and pitch values from string to float**

In [ ]:
center_x = []
ab = new_train_objects['1_x']
for i in range(len(ab)):
    center_x.append(float(ab[i])) 
    


In [ ]:
center_y = []
ab = new_train_objects['1_y']
for i in range(len(ab)):
    center_y.append(float(ab[i])) 

In [ ]:
center_z = []
ab = new_train_objects['1_z']
for i in range(len(ab)):
    center_z.append(float(ab[i])) 


In [ ]:
pitch = []
ab = new_train_objects['1_pitch']
for i in range(len(ab)):
    pitch.append(float(ab[i])) 


In [ ]:
roll = []
ab = new_train_objects['1_roll']
for i in range(len(ab)):
    roll.append(float(ab[i])) 
    

**Distribution of center_z**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(center_z, color='red', ax=ax).set_title('center_z', fontsize=16)
plt.xlabel('center_z', fontsize=15)
plt.show()

**Distribution of pitch**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(pitch, color='black', ax=ax).set_title('pitch', fontsize=16)
plt.xlabel('pitch', fontsize=15)
plt.show()

**Distribution of Roll**

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(roll, color='purple', ax=ax).set_title('Roll', fontsize=16)
plt.xlabel('Roll', fontsize=15)
plt.show()

**we can see inconsistency in pitch and roll distribution**

# Distributions of center_x and center_y


In [ ]:

fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(center_x, color='darkorange', ax=ax).set_title('center_x and center_y', fontsize=16)
sns.distplot(center_y, color='purple', ax=ax).set_title('center_x and center_y', fontsize=16)
plt.xlabel('center_x and center_y', fontsize=15)
plt.show()

from the diagram above,

**darkorange = center_x**

**purple = center_y**

**Distribution of yaw**

In [ ]:
yaw = []
ab = new_train_objects['1_yaw']
for i in range(len(ab)):
    yaw.append(float(ab[i])) 

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
sns.distplot(yaw, color='darkgreen', ax=ax).set_title('yaw', fontsize=16)
plt.xlabel('yaw', fontsize=15)
plt.show()

In [ ]:
#from car_models_json file
car_models = ('guangqi-chuanqi-GS4-2015.json', 'sikeda-jingrui.json', 'dazhong-SUV.json', 
'biyadi-qin.json', 'jipu-3.json', 'qirui-ruihu.json', 'leikesasi.json', 'biaozhi-liangxiang.json',
'rongwei-750.json', 'fengtian-puladuo-06.json', 'changanbenben.json', 'changan-cs5.json',
'lingmu-SX4-2012.json', 'biaozhi-508.json', 'linken-SUV.json', 'Skoda_Fabia-2011.json', 'dazhong.json',
'yiqi-benteng-b50.json', 'baoma-X5.json', 'jili-boyue.json', 'biyadi-2x-F0.json', 'biaozhi-3008.json',
'qiya.json', 'sikeda-SUV.json', '036-CAR01.json', 'fengtian-liangxiang.json', 'jilixiongmao-2015.json',
'fengtian-SUV-gai.json', 'dongfeng-xuetielong-C6.json', 'fengtian-MPV.json', 'xiandai-suonata.json',
'baoma-530.json', 'baojun-510.json', 'baoma-330.json', 'biyadi-F3.json', '037-CAR02.json',
'lingmu-aotuo-2009.json', 'baojun-310-2017.json', 'fengtian-weichi-2006.json', 'bieke-yinglang-XT.json', 
'biaozhi-408.json', 'lufeng-X8.json', 'yingfeinidi-qx80.json', 'changcheng-H6-2016.json', 'fute.json', 
'mazida-6-2015.json', 'bieke.json', 'oubao.json', 'bieke-kaiyue.json', 'feiyate.json', 'haima-3.json',
'baoshijie-kayan.json', 'yingfeinidi-SUV.json', 'dongfeng-yulong-naruijie.json', 
'dongfeng-fengguang-S560.json', 'dazhongmaiteng.json', 'lingmu-swift.json', 'benchi-ML500.json', 
'sanling-oulande.json', 'bentian-fengfan.json', 'baoshijie-paoche.json', 'MG-GT-2015.json', 
'beiqi-huansu-H3.json', 'biyadi-tang.json', 'jianghuai-ruifeng-S3.json', 'rongwei-RX5.json', 
'dongnan-V3-lingyue-2011.json', 'kaidilake-CTS.json', '019-SUV.json', 'benchi-SUR.json',
'dongfeng-fengxing-SX6.json', 'dihao-EV.json', 'aodi-a6.json', 'xiandai-i25-2016.json', 
'dongfeng-DS5.json', 'changan-CS35-2012.json', 'benchi-GLK-300.json', 'supai-2016.json', 'aodi-Q7-SUV.json')


**There are actually 3 types of car models provided in car_models_json file.**

In [ ]:
suv = twox = threex = 0
for i in range(len(car_models)):
    with open('../input/pku-autonomous-driving/car_models_json/'+car_models[i]) as json_file:
        data = json.load(json_file)
        print('car_type: '+data['car_type'])
        if(data['car_type'] == 'SUV'):
            suv+=1
        elif(data['car_type'] == '2x'):
            twox+=1
        else:
            threex+=1
        


In [ ]:
print('Total SUV:', suv)       
print('Total 2x:', twox)  
print('Total 3x:', threex)  

**Lets Visualize one SUV model,one 2x model and one 3x model car below**

In [ ]:
#taken from here  : https://www.kaggle.com/ebouteillon/load-a-3d-car-model

with open('../input/pku-autonomous-driving/car_models_json/'+car_models[0]) as json_file:
    data = json.load(json_file)
    vertices = np.array(data['vertices'])
    triangles = np.array(data['faces']) - 1
    plt.figure(figsize=(20,10))
    ax = plt.axes(projection='3d')
    ax.set_title('car_type: '+data['car_type'])
    ax.set_xlim([-3, 3])
    ax.set_ylim([-3, 3])
    ax.set_zlim([0, 3])
    ax.plot_trisurf(vertices[:,0], vertices[:,2], triangles, -vertices[:,1], shade=True, color='green')

with open('../input/pku-autonomous-driving/car_models_json/'+car_models[1]) as json_file:
    data = json.load(json_file)
    vertices = np.array(data['vertices'])
    triangles = np.array(data['faces']) - 1
    plt.figure(figsize=(20,10))
    ax = plt.axes(projection='3d')
    ax.set_title('car_type: '+data['car_type'])
    ax.set_xlim([-3, 3])
    ax.set_ylim([-3, 3])
    ax.set_zlim([0, 3])
    ax.plot_trisurf(vertices[:,0], vertices[:,2], triangles, -vertices[:,1], shade=True, color='red')
    
with open('../input/pku-autonomous-driving/car_models_json/'+car_models[3]) as json_file:
    data = json.load(json_file)
    vertices = np.array(data['vertices'])
    triangles = np.array(data['faces']) - 1
    plt.figure(figsize=(20,10))
    ax = plt.axes(projection='3d')
    ax.set_title('car_type: '+data['car_type'])
    ax.set_xlim([-3, 3])
    ax.set_ylim([-3, 3])
    ax.set_zlim([0, 3])
    ax.plot_trisurf(vertices[:,0], vertices[:,2], triangles, -vertices[:,1], shade=True, color='yellow')


In [ ]:
print(os.listdir(PATH + 'camera'))

**Lets read what inside that camera folder**

inside camera folder we have got camera_intrinsic.txt file,we will read that file in cell below

In [ ]:
file=open(PATH + '/camera/camera_intrinsic.txt', "r").read().split('\n')
for i in range(len(file)):
    print(file[i])

In [ ]:
#Taken From : https://www.kaggle.com/hocop1/centernet-baseline
# From camera.zip
camera_matrix = np.array([[2304.5479, 0,  1686.2379],
                          [0, 2305.8757, 1354.9849],
                          [0, 0, 1]], dtype=np.float32)
camera_matrix_inv = np.linalg.inv(camera_matrix)

def str2coords(s, names=['id', 'yaw', 'pitch', 'roll', 'x', 'y', 'z']):
    '''
    Input:
        s: PredictionString (e.g. from train dataframe)
        names: array of what to extract from the string
    Output:
        list of dicts with keys from `names`
    '''
    coords = []
    for l in np.array(s.split()).reshape([-1, 7]):
        coords.append(dict(zip(names, l.astype('float'))))
        if 'id' in coords[-1]:
            coords[-1]['id'] = int(coords[-1]['id'])
    return coords

def get_img_coords(s):
    '''
    Input is a PredictionString (e.g. from train dataframe)
    Output is two arrays:
        xs: x coordinates in the image
        ys: y coordinates in the image
    '''
    coords = str2coords(s)
    xs = [c['x'] for c in coords]
    ys = [c['y'] for c in coords]
    zs = [c['z'] for c in coords]
    P = np.array(list(zip(xs, ys, zs))).T
    img_p = np.dot(camera_matrix, P).T
    img_p[:, 0] /= img_p[:, 2]
    img_p[:, 1] /= img_p[:, 2]
    img_xs = img_p[:, 0]
    img_ys = img_p[:, 1]
    img_zs = img_p[:, 2] # z = Distance from the camera
    return img_xs, img_ys

plt.figure(figsize=(20,20))
plt.imshow(imread(PATH + 'train_images/' + train['ImageId'][1] + '.jpg'))
plt.scatter(*get_img_coords(train['PredictionString'][1]), color='red', s=100);

**What's Inside train.csv?**

In [ ]:
a = train
a.head()


**We understand ImageId column contains id of each image but why PredictionString?**

Lets take a look at first row of PredictionString 

In [ ]:
a.PredictionString[0]

**PredictionString containing a collection of poses and confidence scores.**

# FOR Understanding data please check [this link](https://www.kaggle.com/c/pku-autonomous-driving/data)

**Visualizing first 5 train images from train_images directory**

In [ ]:
i = 0
for img in glob.glob(PATH + "train_images/*.jpg"):
    i+=1
    if(i==6):
        break
    img = cv2.imread(img)
    plt.figure(figsize=(15,8))
    plt.imshow(img);

**Visualizing first 5 train_masks from train_masks directory**

In [ ]:
i = 0
for img in glob.glob(PATH + "train_masks/*.jpg"):
    i+=1
    if(i==6):
        break
    img = cv2.imread(img)
    plt.figure(figsize=(15,8))
    plt.imshow(img);

**Visualizing first 10 test_images from test_images directory**

In [ ]:
i = 0
for img in glob.glob(PATH + "test_images/*.jpg"):
    i+=1
    if(i==11):
        break
    img = cv2.imread(img)
    plt.figure(figsize=(15,8))
    plt.imshow(img);

**Visualizing first 10 test_masks from test_masks directory**

In [ ]:
i = 0
for img in glob.glob(PATH + "test_masks/*.jpg"):
    i+=1
    if(i==11):
        break
    img = cv2.imread(img)
    plt.figure(figsize=(15,8))
    plt.imshow(img);

In [ ]:
train = pd.read_csv(PATH + 'train.csv')
test = pd.read_csv(PATH + 'sample_submission.csv')
bad_list = ['ID_1a5a10365',
'ID_1db0533c7',
'ID_53c3fe91a',
'ID_408f58e9f',
'ID_4445ae041',
'ID_bb1d991f6',
'ID_c44983aeb',
'ID_f30ebe4d4']
train = train.loc[~train['ImageId'].isin(bad_list)]
# From camera.zip
camera_matrix = np.array([[2304.5479, 0,  1686.2379],
                          [0, 2305.8757, 1354.9849],
                          [0, 0, 1]], dtype=np.float32)
camera_matrix_inv = np.linalg.inv(camera_matrix)

train.head()

In [ ]:
def imread(path, fast_mode=False):
    img = cv2.imread(path)
    if not fast_mode and img is not None and len(img.shape) == 3:
        img = np.array(img[:, :, ::-1])
    return img

img = imread(PATH + 'train_images/ID_8a6e65317' + '.jpg')
IMG_SHAPE = img.shape

plt.figure(figsize=(15,8))
plt.imshow(img);

**REF : https://www.kaggle.com/hocop1/centernet-baseline**

In [ ]:
def rotate(x, angle):
    x = x + angle
    x = x - (x + np.pi) // (2 * np.pi) * 2 * np.pi
    return x


In [ ]:
IMG_WIDTH = 2048
IMG_HEIGHT = 512
MODEL_SCALE = 8

def _regr_preprocess(regr_dict, flip=False):
    if flip:
        for k in ['x', 'pitch', 'roll']:
            regr_dict[k] = -regr_dict[k]
    for name in ['x', 'y', 'z']:
        regr_dict[name] = regr_dict[name] / 100
    regr_dict['roll'] = rotate(regr_dict['roll'], np.pi)
    regr_dict['pitch_sin'] = sin(regr_dict['pitch'])
    regr_dict['pitch_cos'] = cos(regr_dict['pitch'])
    regr_dict.pop('pitch')
    regr_dict.pop('id')
    return regr_dict

def _regr_back(regr_dict):
    for name in ['x', 'y', 'z']:
        regr_dict[name] = regr_dict[name] * 100
    regr_dict['roll'] = rotate(regr_dict['roll'], -np.pi)
    
    pitch_sin = regr_dict['pitch_sin'] / np.sqrt(regr_dict['pitch_sin']**2 + regr_dict['pitch_cos']**2)
    pitch_cos = regr_dict['pitch_cos'] / np.sqrt(regr_dict['pitch_sin']**2 + regr_dict['pitch_cos']**2)
    regr_dict['pitch'] = np.arccos(pitch_cos) * np.sign(pitch_sin)
    return regr_dict

def preprocess_image(img, flip=False):
    img = img[img.shape[0] // 2:]
    bg = np.ones_like(img) * img.mean(1, keepdims=True).astype(img.dtype)
    bg = bg[:, :img.shape[1] // 6]
    img = np.concatenate([bg, img, bg], 1)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    if flip:
        img = img[:,::-1]
    return (img / 255).astype('float32')

def get_mask_and_regr(img, labels, flip=False):
    mask = np.zeros([IMG_HEIGHT // MODEL_SCALE, IMG_WIDTH // MODEL_SCALE], dtype='float32')
    regr_names = ['x', 'y', 'z', 'yaw', 'pitch', 'roll']
    regr = np.zeros([IMG_HEIGHT // MODEL_SCALE, IMG_WIDTH // MODEL_SCALE, 7], dtype='float32')
    coords = str2coords(labels)
    xs, ys = get_img_coords(labels)
    for x, y, regr_dict in zip(xs, ys, coords):
        x, y = y, x
        x = (x - img.shape[0] // 2) * IMG_HEIGHT / (img.shape[0] // 2) / MODEL_SCALE
        x = np.round(x).astype('int')
        y = (y + img.shape[1] // 6) * IMG_WIDTH / (img.shape[1] * 4/3) / MODEL_SCALE
        y = np.round(y).astype('int')
        if x >= 0 and x < IMG_HEIGHT // MODEL_SCALE and y >= 0 and y < IMG_WIDTH // MODEL_SCALE:
            mask[x, y] = 1
            regr_dict = _regr_preprocess(regr_dict, flip)
            regr[x, y] = [regr_dict[n] for n in sorted(regr_dict)]
    if flip:
        mask = np.array(mask[:,::-1])
        regr = np.array(regr[:,::-1])
    return mask, regr

In [ ]:
class CarDataset(Dataset):
    """Car dataset."""

    def __init__(self, dataframe, root_dir, training=True, transform=True):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.training = training
        if self.transform :
            if self.training :
                self.aug = Compose([
                    
                            #RandomGamma(p=0.2),
                            #HorizontalFlip(),

                            Cutout(num_holes=20,p=0.1),
                        
                            ])
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get image name
        idx, labels = self.df.values[idx]
        img_name = self.root_dir.format(idx)
        
        # Read image
        img0 = imread(img_name, True)
        img = preprocess_image(img0)
        img = np.rollaxis(img, 2, 0)
        # Get mask and regression maps
        if self.training:
            mask, regr = get_mask_and_regr(img0, labels)
            regr = np.rollaxis(regr, 2, 0)
        else:
            mask, regr = 0, 0
        if self.transform:
            augmented = self.aug(image=img, mask=mask,regr=regr)
            img, mask,regr = augmented['image'],augmented['mask'] ,augmented['regr']
            
        return [img, mask, regr]

In [ ]:
train_images_dir = PATH + 'train_images/{}.jpg'
test_images_dir = PATH + 'test_images/{}.jpg'

df_train, df_dev = train_test_split(train, test_size=0.1, random_state=63)
df_test = test

# Create dataset objects
train_dataset = CarDataset(df_train, train_images_dir)
dev_dataset = CarDataset(df_dev, train_images_dir)
test_dataset = CarDataset(df_test, test_images_dir)

# Without HorizontalFlip

In [ ]:
img, mask, regr = dev_dataset[10]

plt.figure(figsize=(10,10))
plt.imshow(np.rollaxis(img, 0, 3))
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(mask)
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(regr[-2])
plt.show()

In [ ]:
class CarDataset(Dataset):
    """Car dataset."""

    def __init__(self, dataframe, root_dir, training=True, transform=True):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.training = training
        if self.transform :
            if self.training :
                self.aug = Compose([
                    
                            #RandomGamma(p=0.2),
                            HorizontalFlip(),

                         
                        
                            ])
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get image name
        idx, labels = self.df.values[idx]
        img_name = self.root_dir.format(idx)
        
        # Read image
        img0 = imread(img_name, True)
        img = preprocess_image(img0)
        img = np.rollaxis(img, 2, 0)
        # Get mask and regression maps
        if self.training:
            mask, regr = get_mask_and_regr(img0, labels)
            regr = np.rollaxis(regr, 2, 0)
        else:
            mask, regr = 0, 0
        if self.transform:
            augmented = self.aug(image=img, mask=mask,regr=regr)
            img, mask,regr = augmented['image'],augmented['mask'] ,augmented['regr']
            
        return [img, mask, regr]

In [ ]:
train_images_dir = PATH + 'train_images/{}.jpg'
test_images_dir = PATH + 'test_images/{}.jpg'

df_train, df_dev = train_test_split(train, test_size=0.1, random_state=63)
df_test = test

# Create dataset objects
train_dataset = CarDataset(df_train, train_images_dir)
dev_dataset = CarDataset(df_dev, train_images_dir)
test_dataset = CarDataset(df_test, test_images_dir)

# After Applying HorizontalFlip

In [ ]:
img, mask, regr = dev_dataset[10]

plt.figure(figsize=(10,10))
plt.imshow(np.rollaxis(img, 0, 3))
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(mask)
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(regr[-2])
plt.show()

# lets try GaussNoise 

In [ ]:
import albumentations as A
A.MultiplicativeNoise()

In [ ]:


class CarDataset(Dataset):
    """Car dataset."""

    def __init__(self, dataframe, root_dir, training=True, transform=True):
        self.df = dataframe
        self.root_dir = root_dir
        self.transform = transform
        self.training = training
        if self.transform :
            if self.training :
                self.aug = Compose([
                    
                            #RandomGamma(p=0.2),
                            #A.ISONoise(),
                            A.IAAPerspective(p=1,keep_size=False),
                            A.GaussNoise()
                         
                        
                            ])
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # Get image name
        idx, labels = self.df.values[idx]
        img_name = self.root_dir.format(idx)
        
        # Read image
        img0 = imread(img_name, True)
        img = preprocess_image(img0)
        img = np.rollaxis(img, 2, 0)
        # Get mask and regression maps
        if self.training:
            mask, regr = get_mask_and_regr(img0, labels)
            regr = np.rollaxis(regr, 2, 0)
        else:
            mask, regr = 0, 0
        if self.transform:
            augmented = self.aug(image=img, mask=mask,regr=regr)
            img, mask,regr = augmented['image'],augmented['mask'] ,augmented['regr']
            
        return [img, mask, regr]
    
    
train_images_dir = PATH + 'train_images/{}.jpg'
test_images_dir = PATH + 'test_images/{}.jpg'

df_train, df_dev = train_test_split(train, test_size=0.1, random_state=63)
df_test = test

# Create dataset objects
train_dataset = CarDataset(df_train, train_images_dir)
dev_dataset = CarDataset(df_dev, train_images_dir)
test_dataset = CarDataset(df_test, test_images_dir)

img, mask, regr = dev_dataset[10]

plt.figure(figsize=(20,10))
plt.imshow(np.rollaxis(img, 0, 3))
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(mask)
plt.show()

plt.figure(figsize=(8,8))
plt.imshow(regr[-2])
plt.show()

In [ ]:
train_dataset = CarDataset(df_train, train_images_dir, transform=False)
BATCH_SIZE = 2

# Create data generators - they will produce batches
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2,drop_last=True)
dev_loader = DataLoader(dataset=dev_dataset, batch_size=2, shuffle=False, num_workers=2,drop_last=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

IF YOU SPOT ANY ERROR,PLEASE LET ME KNOW IN THE COMMENT SECTION AND IT WILL BE HIGHLY APPRECIATED

**Thanks for reading**